In [1]:
# import sys
# !{sys.executable} -m pip install altair vega_datasets

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

In [3]:
# vaccination_columns = ['date', 'iso_code', 'continent', 'location', 'human_development_index', 'extreme_poverty',
#                     'gdp_per_capita', 'new_vaccinations', 'new_vaccinations_smoothed', 'new_vaccinations_smoothed_per_million',
#                     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred', 'people_vaccinated', 'people_vaccinated_per_hundred',
#                     'population', 'population_density', 'total_boosters', 'total_boosters_per_hundred', 'total_vaccinations', 'total_vaccinations_per_hundred']

financial_columns = ['location', 'human_development_index', 'extreme_poverty',
                    'gdp_per_capita', 'population', 'population_density']
vaccination_columns = ['date', 'location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated']

financial_data = pd.read_csv('../datasets/owid-covid-data.csv', usecols=financial_columns)
covid_data = pd.read_csv('../datasets/owid-covid-data.csv', usecols=vaccination_columns)
countries = pd.read_csv('../datasets/locations.csv')['location'].to_list()
covid_data.head()

,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,population
0,Afghanistan,2020-02-24,NaN,NaN,NaN,39835428.0
1,Afghanistan,2020-02-25,NaN,NaN,NaN,39835428.0
2,Afghanistan,2020-02-26,NaN,NaN,NaN,39835428.0
3,Afghanistan,2020-02-27,NaN,NaN,NaN,39835428.0
4,Afghanistan,2020-02-28,NaN,NaN,NaN,39835428.0


In [4]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(owid_covid_data, title="Pandas Profiling Report", explorative=True)
# profile.to_widgets()

In [5]:
a = covid_data.dropna(subset=["total_vaccinations", "people_vaccinated", "people_fully_vaccinated"]).drop_duplicates(subset=["location"], keep="last")
from IPython.display import display, HTML

display(HTML(a.to_html()))

,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,population
543,Afghanistan,2021-08-20,1.201286e+06,7.705420e+05,4.307440e+05,3.983543e+07
1210,Africa,2021-10-15,1.710956e+08,1.048424e+08,6.883170e+07,1.373486e+09
1807,Albania,2021-10-13,1.816580e+06,9.707030e+05,8.458770e+05,2.872934e+06
2388,Algeria,2021-09-25,1.408292e+07,6.017036e+06,4.032942e+06,4.461663e+07
2982,Andorra,2021-09-26,1.020320e+05,5.431200e+04,4.772000e+04,7.735400e+04
3575,Angola,2021-10-14,4.448653e+06,3.158501e+06,1.290152e+06,3.393361e+07
3830,Anguilla,2021-10-15,1.870500e+04,9.548000e+03,9.157000e+03,1.512500e+04
4410,Antigua and Barbuda,2021-10-13,9.984600e+04,5.475800e+04,4.508800e+04,9.872800e+04
5066,Argentina,2021-10-15,5.510371e+07,3.087864e+07,2.430886e+07,4.560582e+07
5656,Armenia,2021-10-11,5.142410e+05,3.440290e+05,1.702120e+05,2.968128e+06


In [6]:
only_countries = a[a['location'].isin(countries)]
only_countries

,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,population
543,Afghanistan,2021-08-20,1201286.0,770542.0,430744.0,39835428.0
1807,Albania,2021-10-13,1816580.0,970703.0,845877.0,2872934.0
2388,Algeria,2021-09-25,14082920.0,6017036.0,4032942.0,44616626.0
2982,Andorra,2021-09-26,102032.0,54312.0,47720.0,77354.0
3575,Angola,2021-10-14,4448653.0,3158501.0,1290152.0,33933611.0
...,...,...,...,...,...,...
121439,Vietnam,2021-10-14,59003239.0,41811429.0,17191810.0,98168829.0
121644,Wallis and Futuna,2021-10-12,11009.0,5749.0,5260.0,11094.0
122813,Yemen,2021-09-27,356173.0,308025.0,48148.0,30490639.0
123370,Zambia,2021-09-07,602996.0,311049.0,291947.0,18920657.0


In [7]:
only_continents = a[~a['location'].isin([*countries, 'World'])]                                
only_continents.sort_values(by=['total_vaccinations'], ascending=False)

,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,population
6494,Asia,2021-10-15,4.480253e+09,2.541010e+09,1.841308e+09,4.678445e+09
37831,Europe,2021-10-15,8.225248e+08,4.325363e+08,4.016609e+08,7.489630e+08
82449,North America,2021-10-15,6.491530e+08,3.539533e+08,2.947244e+08,5.965813e+08
38463,European Union,2021-10-15,5.787278e+08,3.047014e+08,2.865587e+08,4.471899e+08
103487,South America,2021-10-15,4.743950e+08,2.799747e+08,1.995274e+08,4.342601e+08
1210,Africa,2021-10-15,1.710956e+08,1.048424e+08,6.883170e+07,1.373486e+09
84531,Oceania,2021-10-15,4.087258e+07,2.316535e+07,1.770722e+07,4.321995e+07


In [8]:
HTML(only_continents.to_html())

,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,population
1210,Africa,2021-10-15,1.710956e+08,1.048424e+08,6.883170e+07,1.373486e+09
6494,Asia,2021-10-15,4.480253e+09,2.541010e+09,1.841308e+09,4.678445e+09
37831,Europe,2021-10-15,8.225248e+08,4.325363e+08,4.016609e+08,7.489630e+08
38463,European Union,2021-10-15,5.787278e+08,3.047014e+08,2.865587e+08,4.471899e+08
82449,North America,2021-10-15,6.491530e+08,3.539533e+08,2.947244e+08,5.965813e+08
84531,Oceania,2021-10-15,4.087258e+07,2.316535e+07,1.770722e+07,4.321995e+07
103487,South America,2021-10-15,4.743950e+08,2.799747e+08,1.995274e+08,4.342601e+08


In [9]:

# alt.Chart(only_continents).mark_bar().encode(
#   x='location',
#   y='total_vaccinations',
#   color='Origin'
# )

# alt.Chart(cars).mark_bar().encode(
#   x='mean(Miles_per_Gallon)',
#   y='Origin',
#   color='Origin'
# )

In [12]:
import altair as alt

alt.Chart(only_continents).mark_bar().encode(
    y = alt.Y("location:O", title="Country"),
    x = alt.X("total_vaccinations:Q", title="Total number of vaccinations administered"),
    # The highlight will be set on the result of a conditional statement
    color=alt.condition(
        alt.datum.location == 'Asia',  # If the year is 1810 this test returns True,
        alt.value('orange'),     # which sets the bar orange.
        alt.value('steelblue')   # And if it's not true it sets the bar steelblue.
    )
).properties(width=600)


alt.Chart(...)

In [13]:
source = only_countries.sort_values(by=['total_vaccinations'], ascending=False).iloc[0:20]



bars = alt.Chart(source).mark_bar().encode(
    x="total_vaccinations:Q",
    y='location:O',
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='total_vaccinations:Q'
)

(bars + text).properties(height=300)
print(0)

0
